In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader , Subset
from torchvision.transforms import v2
import torchvision.transforms.functional as F
from PIL import Image
import os
import random
import torch.optim as optim



In [12]:
# config.py

DATA_DIR = "/home/aman/code/CV/skin_infec_detect/data/IMG_CLASSES"
DEVICE = "cuda"
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 30
CLASSES = []
DEPTH = 121
GROWTH_RATE = 32
REDUCTION_RATE = 0.5
DROP_RATE = 0.3
LOSS_FN = nn.BCEWithLogitsLoss()
CLASS_WEIGHTS = []
DATA_SIZE = sum([len(os.listdir(os.path.join(DATA_DIR , i))) for i in os.listdir(DATA_DIR)])


# tranformation

# base transformation

base_transform = v2.Compose(
    [
        v2.Resize((256,256))
    ]
)

# flipping 
trans_flip = v2.Compose(
    [
        v2.RandomHorizontalFlip(p=0.8)
    ]
)

new_h = random.randrange(200 ,256)
new_w = random.randrange(200 , 256)

trans_zoom = v2.Compose(
    [
        v2.RandomCrop(size=(new_h , new_w))
    ]
)

# rotation 
trans_rotate = v2.Compose(
    [
        v2.RandomRotation(degrees=(-10 , 10))
    ]
)

def adjust_brightness(image):

    factor = random.uniform(0.5 , 1.9)

    return F.adjust_brightness(image , factor)

TRANS_ALL = [trans_flip , trans_zoom , trans_rotate , adjust_brightness]



In [13]:
# utils.py

def get_classes(data_dir):
    
    for i in os.listdir(data_dir):
        if i not in CLASSES:
            CLASSES.append(i)

def apply_transformations(img , image_dir , image_name):
    print("I am here")
    img = base_transform(img)
    for trans in TRANS_ALL:

        trans_img = trans(img)
        filename = f"{image_dir}_{trans}_{image_name}"
        filepath = os.path.join(image_dir , filename)
        print(filepath)
        trans_img.save(filepath)
        
def custom_weights():
    # weights = total samples / no. of samples in that grp
    
    for i in range(len(CLASSES)):
        weight_ = DATA_SIZE / len(os.listdir(os.path.join(DATA_DIR , CLASSES[i])))
        CLASS_WEIGHTS.append(weight_)





In [ ]:
# data_aug.py

# for i in CLASSES:
#     print("hi")
#     image_dir = os.path.join(DATA_DIR , i)

#     for image_name in os.listdir(image_dir):

#         image_path = os.path.join(image_dir , image_name)

#         img = Image.open(image_path)

#         apply_transformations(img , image_dir=image_dir , image_name=image_name)


In [7]:

folder_dir = "/home/aman/code/CV/skin_infec_detect/data/IMG_CLASSES"
s = 0
print(len(os.listdir(folder_dir)))
for i in os.listdir(folder_dir):

   
    s += len(os.listdir(os.path.join(folder_dir , i)))
    print(f"{i} , {len(os.listdir(os.path.join(folder_dir , i)))}")

s

14
Atopic Dermatitis , 6285
Vascular lesion , 500
Melanoma , 15700
Squamous cell carcinoma , 500
Dermatofibroma , 500
Eczema , 8385
Seborrheic Keratoses and other Benign Tumors , 9235
Benign Keratosis-like Lesions , 10395
Tinea Ringworm Candidiasis and other Fungal Infections , 8510
Warts Molluscum and other Viral Infections , 10515
Psoriasis pictures Lichen Planus , 10275
Melanocytic Nevi , 39850
Basal Cell Carcinoma , 16615
Actinic keratosis , 500


137765

In [17]:
# model.py



28